In [ ]:
## importing libraries
import gradio as gr
from openai import OpenAI
import base64
from dotenv import load_dotenv
from PIL import Image
import io
import os


In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [15]:
# Function to convert image to base64
def encode_image(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

# Function to analyze image + text with GPT-4o
def analyze_image_text(image, text_input):
    if image is None:
        return "Please upload an image."

    # Convert image to base64
    img_base64 = encode_image(image)

    # Construct OpenAI request
    messages = [{"role": "system", "content": "You are an AI that analyzes images and text."}]
    
    if text_input:
        messages.append({"role": "user", "content": text_input})

    messages.append({"role": "user", "content": {"type": "image_url", "image_url": f"data:image/png;base64,{img_base64}"}})

    # Call OpenAI API
    openai.api_key = openai_api_key
    response =openai.chat.completions.create(model=MODEL, messages=messages)

    return response["choices"][0]["message"]["content"]

In [16]:
# Gradio UI
interface = gr.Interface(
    fn=analyze_image_text,
    inputs=[
        gr.Image(type="pil"),  # Upload image
        gr.Textbox(label="Enter additional text for analysis (optional)"),
    ],
    outputs="text",
    title="📸 Multimodal AI: Image & Text Analyzer",
    description="Upload an image and enter text (optional). The AI will analyze both and provide insights.",
)

In [17]:
# Launch Gradio app
interface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\rohit\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rohit\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rohit\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2108, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rohit\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1655, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rohit\anaconda3\envs\llms\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return awai